In [3]:
import numpy as np

This is a Sudoku solver written in Python using a class


In [4]:
#this version shows the board each time a number is added.
class sudoku_move: #we introduce a class to create the soduku board display and determine whether a spot on the board can be filled
    def __init__(self, row=-1, col=-1, value=0): #the init method allows us to introduce the attributes about the sudoku when we are using it??????
        self.row = row
        self.col = col
        self.value = value #the number in a particular spot on the board
        
    def display_(self):
        print("A move ["+str(self.row)+","+str(self.col)+"] ="+str(self.value))# this tells the user what value is being put where on the board
   
    def is_valid(self):
        return self.value != 0 # will return True (not 0) or False (0)

class sudoku_board: #new class for the board itself and the functions used to solve them
    def __init__(self,board):
        #self.board = np.zeros((9,9)) here if we want we can adjust so that the board is filled one row at a time by the user
        self.board = board #for now the baord is added as a whole
        self.moves = [] #keeps track of what moves were made so that can undo in the event of an error
    
    def board_check(self):
        for row in range(0,9):
            values_row = np.unique(self.board[row,:])
            for i in values_row[1:]:
                count = 0
                for r in range(0,9):
                    if i == self.board[row,r]:
                        count+=1
                        if count>1:
                            raise RuntimeError("This unsolved sudoku is invalid. There is more that one",i,"in row",row+1)
            #print("The row",row+1,"is satisfactory")   
        #checking the collumns of the board
        for col in range(0,9):
            values_col = np.unique(self.board[:,col])
            for i in values_col[1:]:
                count = 0
                for c in range(0,9):
                    if i == self.board[c,col]:
                        count+=1
                        if count>1:
                            raise RuntimeError("This unsolved sudoku is invalid. There is more that one",i,"in row",row+1)
            #print("The column",col+1,"is satisfactory")
        for row in range(0,7,3):#0,3,6
            for col in range(0,7,3):
                values_grp = np.unique(self.board[row:row+3,col:col+3])#this finds all the different values in each group
                for i in values_grp[1:]:
                    count = 0
                    for r in range(row,row+3):
                        for c in range(col,col+3):
                            if i==self.board[r,c]:
                                count+=1
                                if count>1:
                                    raise RuntimeError("Invalid unsolved sudoku.(Problem in on of the 3x3 grids.)")
                #print("Group",row,":",col,"valid.")
        print("Initial board is valid. Solving will proceed.")
        
      
    def display(self): #displays the board in a more user friendly way
        board_str = self.board.astype(str) #converts the array to strings
        row_sep = '-'*25 #this is to divide the groups and for the border
        for i in range(0,9): #for all the rows
            if i%3 ==0: #every 3 rows
                print(row_sep) 
            row = board_str[i] #print the final product,collumns divided the groups
            print('|'+' '.join(row[0:3])+' | '+' '.join(row[3:6])+' | '+' '.join(row[6:9])+' | ')
        print(row_sep)
    
    def round_3(self,index): #this rounds the row/col number to the next largest multiple of 3 so we have a way to define the group an element is in
        round_float = np.ceil((index+1)/3)*3
        round_int = int(round_float)
        return (round_int) #important to have return here because we will use it
        
    def value(self,row,col): #when we need to handle a single element and find its value
        return self.board[row,col]
    
    def options(self,row,col): #finds the possible values that may be assigned to a spot
        if not self.is_empty(row,col): #if there is a 0 then we need to consider the possible values otherwise
            return self.value(row,col)
        row_value = np.unique(self.board[row,:])
        col_values = np.unique(self.board[:,col])
        row_end = self.round_3(row)
        col_end = self.round_3(col)
        sub_values = np.unique(np.unique(self.board[row_end-3:row_end,col_end-3:col_end]))
        all_values = np.concatenate((row_value,col_values,sub_values),axis = None)
        return [value for value in range(0,10) if value not in all_values] #by the end of this regardless we have at least one value for each element 

    def is_empty(self, row, col): # True if value is a 0, False if filled
        return self.value(row,col) == 0

    def is_solved(self): # will return False until all the spots are filled
        for row in range(0,9):
            for col in range(0,9):
                if self.is_empty(row,col):
                    return False
        return True
    
    def propose(self): # this goes through all the elements on the board and if there is only one possible value then it returns it
        for row in range(0,9):
            for col in range(0,9):
                if self.is_empty(row,col):
                    options = self.options(row,col)
                    if len(options) == 1:
                        return sudoku_move(row,col,options[0])
        return sudoku_move()
    
    def move(self,move): # this is where the possible move that is proposed is checked and when it's cleared then it is applied
        if not self.is_empty(move.row, move.col): #this is exception handling where if the value proposed doesn't work, error shows
            raise RuntimeError("Move already completed")
        if not move.value in self.options(move.row,move.col):
            raise RuntimeError("Invalid move")
        self.moves.append(move) #once we're sure that the value is ok, now it is added to the list of moves
        self.board[move.row, move.col] = move.value      # and the move is completed  
        
    def undo(self): #undoes a move
        if len(self.moves) == 0:
            raise RuntimeError("No moves to undo")
        move = self.moves.pop(-1)
        self.board[move.row,move.col] = 0
        return move
        
    def solve(self): # this is what puts all the functions together to complete the sudoku
        game.display()# shows the intial board
        game.board_check()
        move = game.propose() #finds the instance where there's only one possible value and returns it
        while move.is_valid(): # this loop allows the whole board to be filled
            game.move(move) #the move is comleted
            game.display() #the updated board is displayed
            move = game.propose() # next move is proposed



In [57]:
#This version will tell the user what moves are done, but won't show the board each time (more compact)
class sudoku_move: #we introduce a class to create the soduku board display and determine whether a spot on the board can be filled
    def __init__(self, row=-1, col=-1, value=0): #the init method allows us to introduce the attributes about the sudoku when we are using it??????
        self.row = row
        self.col = col
        self.value = value #the number in a particular spot on the board
        
    def display_(self):
        print("A move ["+str(self.row)+","+str(self.col)+"] ="+str(self.value))# this tells the user what value is being put where on the board
   
    def is_valid(self):
        return self.value != 0 # will return True (not 0) or False (0)

class sudoku_board: #new class for the board itself and the functions used to solve them
    def __init__(self,board):
        #self.board = np.zeros((9,9)) here if we want we can adjust so that the board is filled one row at a time by the user
        self.board = board #for now the baord is added as a whole
        self.moves = [] #keeps track of what moves were made so that can undo in the event of an error
    
    def board_check(self):
        for row in range(0,9):
            values_row = np.unique(self.board[row,:])
            for i in values_row[1:]:
                count = 0
                for r in range(0,9):
                    if i == self.board[row,r]:
                        count+=1
                        if count>1:
                            raise RuntimeError("This unsolved sudoku is invalid. There is more that one",i,"in row",row+1)
            #print("The row",row+1,"is satisfactory")   
        #checking the collumns of the board
        for col in range(0,9):
            values_col = np.unique(self.board[:,col])
            for i in values_col[1:]:
                count = 0
                for c in range(0,9):
                    if i == self.board[c,col]:
                        count+=1
                        if count>1:
                            raise RuntimeError("This unsolved sudoku is invalid. There is more that one",i,"in row",row+1)
            #print("The column",col+1,"is satisfactory")
        for row in range(0,7,3):#0,3,6
            for col in range(0,7,3):
                values_grp = np.unique(self.board[row:row+3,col:col+3])#this finds all the different values in each group
                for i in values_grp[1:]:
                    count = 0
                    for r in range(row,row+3):
                        for c in range(col,col+3):
                            if i==self.board[r,c]:
                                count+=1
                                if count>1:
                                    raise RuntimeError("Invalid unsolved sudoku.(Problem in on of the 3x3 grids.)")
                #print("Group",row,":",col,"valid.")
        print("Initial board is valid. Solving will proceed.")
        
      
    def display(self): #displays the board in a more user friendly way
        board_str = self.board.astype(str) #converts the array to strings
        row_sep = '-'*25 #this is to divide the groups and for the border
        for i in range(0,9): #for all the rows
            if i%3 ==0: #every 3 rows
                print(row_sep) 
            row = board_str[i] #print the final product,collumns divided the groups
            print('|'+' '.join(row[0:3])+' | '+' '.join(row[3:6])+' | '+' '.join(row[6:9])+' | ')
        print(row_sep)
    
    def round_3(self,index): #this rounds the row/col number to the next largest multiple of 3 so we have a way to define the group an element is in
        round_float = np.ceil((index+1)/3)*3
        round_int = int(round_float)
        return (round_int) #important to have return here because we will use it
        
    def value(self,row,col): #when we need to handle a single element and find its value
        return self.board[row,col]
    
    def options(self,row,col): #finds the possible values that may be assigned to a spot
        if not self.is_empty(row,col): #if there is a 0 then we need to consider the possible values otherwise
            return self.value(row,col)
        row_value = np.unique(self.board[row,:])
        col_values = np.unique(self.board[:,col])
        row_end = self.round_3(row)
        col_end = self.round_3(col)
        sub_values = np.unique(np.unique(self.board[row_end-3:row_end,col_end-3:col_end]))
        all_values = np.concatenate((row_value,col_values,sub_values),axis = None)
        return [value for value in range(0,10) if value not in all_values] #by the end of this regardless we have at least one value for each element 

    def is_empty(self, row, col): # True if value is a 0, False if filled
        return self.value(row,col) == 0

    def is_solved(self): # will return False until all the spots are filled
        for row in range(0,9):
            for col in range(0,9):
                if self.is_empty(row,col):
                    return False
        return True
    
    def propose(self): # this goes through all the elements on the board and if there is only one possible value then it returns it
        for row in range(0,9):
            for col in range(0,9):
                if self.is_empty(row,col):
                    options = self.options(row,col)
                    if len(options) == 1:
                        return sudoku_move(row,col,options[0])
        return sudoku_move()
    
    def move(self,move): # this is where the possible move that is proposed is checked and when it's cleared then it is applied
        if not self.is_empty(move.row, move.col): #this is exception handling where if the value proposed doesn't work, error shows
            raise RuntimeError("Move already completed")
        if not move.value in self.options(move.row,move.col):
            raise RuntimeError("Invalid move")
        self.moves.append(move) #once we're sure that the value is ok, now it is added to the list of moves
        self.board[move.row, move.col] = move.value #and the move is completed 
        print(move.row,":",move.col,"=",move.value)
        
    def undo(self): #undoes a move
        if len(self.moves) == 0:
            raise RuntimeError("No moves to undo")
        move = self.moves.pop(-1)
        self.board[move.row,move.col] = 0
        return move
        
    def solve(self): # this is what puts all the functions together to complete the sudoku
        game.display()# shows the intial board
        game.board_check()
        move = game.propose() #finds the instance where there's only one possible value and returns it
        while move.is_valid(): # this loop allows the whole board to be filled
            game.move(move) #the move is comleted
            #game.display() #the updated board is displayed
            move = game.propose() # next move is proposed
        game.display()



In [5]:
example = np.array([[5,3,0,7,0,0,0,0,0],[6,0,0,1,9,5,0,0,0],[0,9,8,0,0,0,0,6,0],[8,0,0,0,6,0,0,0,3],[4,0,0,8,0,3,0,0,1],[7,0,0,0,2,0,0,0,6],[0,6,0,0,0,0,2,8,0],[0,0,0,4,1,9,0,0,5],[0,0,0,0,8,0,0,7,9]])

game = sudoku_board(example)
game.solve()


-------------------------
|5 3 0 | 7 0 0 | 0 0 0 | 
|6 0 0 | 1 9 5 | 0 0 0 | 
|0 9 8 | 0 0 0 | 0 6 0 | 
-------------------------
|8 0 0 | 0 6 0 | 0 0 3 | 
|4 0 0 | 8 0 3 | 0 0 1 | 
|7 0 0 | 0 2 0 | 0 0 6 | 
-------------------------
|0 6 0 | 0 0 0 | 2 8 0 | 
|0 0 0 | 4 1 9 | 0 0 5 | 
|0 0 0 | 0 8 0 | 0 7 9 | 
-------------------------
Initial board is valid. Solving will proceed.
-------------------------
|5 3 0 | 7 4 0 | 0 0 0 | 
|6 0 0 | 1 9 5 | 0 0 0 | 
|0 9 8 | 0 0 0 | 0 6 0 | 
-------------------------
|8 0 0 | 0 6 0 | 0 0 3 | 
|4 0 0 | 8 0 3 | 0 0 1 | 
|7 0 0 | 0 2 0 | 0 0 6 | 
-------------------------
|0 6 0 | 0 0 0 | 2 8 0 | 
|0 0 0 | 4 1 9 | 0 0 5 | 
|0 0 0 | 0 8 0 | 0 7 9 | 
-------------------------
-------------------------
|5 3 0 | 7 4 0 | 0 0 0 | 
|6 0 0 | 1 9 5 | 0 0 0 | 
|0 9 8 | 0 3 0 | 0 6 0 | 
-------------------------
|8 0 0 | 0 6 0 | 0 0 3 | 
|4 0 0 | 8 0 3 | 0 0 1 | 
|7 0 0 | 0 2 0 | 0 0 6 | 
-------------------------
|0 6 0 | 0 0 0 | 2 8 0 | 
|0 0 0 | 4 1 9 | 0

-------------------------
|5 3 2 | 7 4 6 | 1 9 8 | 
|6 7 0 | 1 9 5 | 3 4 2 | 
|1 9 8 | 2 3 0 | 5 6 7 | 
-------------------------
|8 5 9 | 0 6 1 | 0 2 3 | 
|4 2 6 | 8 7 3 | 9 0 1 | 
|7 1 3 | 9 2 4 | 8 5 6 | 
-------------------------
|9 6 1 | 3 5 7 | 2 8 4 | 
|2 8 7 | 4 1 9 | 0 3 5 | 
|0 0 0 | 0 8 0 | 0 7 9 | 
-------------------------
-------------------------
|5 3 2 | 7 4 6 | 1 9 8 | 
|6 7 0 | 1 9 5 | 3 4 2 | 
|1 9 8 | 2 3 0 | 5 6 7 | 
-------------------------
|8 5 9 | 0 6 1 | 0 2 3 | 
|4 2 6 | 8 7 3 | 9 0 1 | 
|7 1 3 | 9 2 4 | 8 5 6 | 
-------------------------
|9 6 1 | 3 5 7 | 2 8 4 | 
|2 8 7 | 4 1 9 | 6 3 5 | 
|0 0 0 | 0 8 0 | 0 7 9 | 
-------------------------
-------------------------
|5 3 2 | 7 4 6 | 1 9 8 | 
|6 7 0 | 1 9 5 | 3 4 2 | 
|1 9 8 | 2 3 0 | 5 6 7 | 
-------------------------
|8 5 9 | 0 6 1 | 0 2 3 | 
|4 2 6 | 8 7 3 | 9 0 1 | 
|7 1 3 | 9 2 4 | 8 5 6 | 
-------------------------
|9 6 1 | 3 5 7 | 2 8 4 | 
|2 8 7 | 4 1 9 | 6 3 5 | 
|3 0 0 | 0 8 0 | 0 7 9 | 
------------